### try to make a tag filter - 20250228 CJH
* try to de-noise apriltags if we aren't moving
* will be done directly on the pi
* could be made better if we knew the robot wasn't moving

In [7]:
# make a tag filter that averages tags if they are coming quickly
import numpy as np
import time

class TagFilter:
    def __init__(self, max_dt=1.0, max_averages=10, max_std=0.1):
        self.current_time = time.time()
        self.previous_time = self.current_time
        self.max_dt = max_dt
        self.max_averages = max_averages
        self.max_std = max_std
        self.max_translation = 1000# 0.1 # arbitryary for now, if moving 1 m/s you'd move .02 m/frame at 50FPS 
        self.max_rotation = 1000# 0.05  #  harder - radians and we spin really fast - like 6.28/s so 
        self.is_stable = False
        
        # Use fixed-size arrays initialized with NaN for efficiency
        self.history = {key: np.full(max_averages, np.nan) for key in ["tx", "ty", "tz", "rx", "ry", "rz"]}
        self.index = 0  # Rolling index to track position
        
    def get_values(self, index):
        """Retrieve [tx, ty, tz, rx, ry, rz] at a given history index safely."""
        if self.index == 0:
            raise ValueError("No data has been recorded yet.")
        return [float(self.history[key][index % self.max_averages]) for key in ["tx", "ty", "tz", "rx", "ry", "rz"]]

    def reset(self, tx, ty, tz, rx, ry, rz):
        new_values = np.array([tx, ty, tz, rx, ry, rz])
        for key in self.history:
            self.history[key].fill(np.nan)  # Reset all values to NaN
            self.history[key][0] = new_values[["tx", "ty", "tz", "rx", "ry", "rz"].index(key)]
        self.index = 1  # Reset index to start fresh
    
    def update(self, tx, ty, tz, rx, ry, rz):
        # if we're stable, average the tag values, otherwise reset and return current values
        self.current_time = time.time()
        dt = self.current_time - self.previous_time

        new_values = np.array([tx, ty, tz, rx, ry, rz])

        # If max time delta is exceeded, reset history to only the latest values
        if dt > self.max_dt or self.index == 0:
            self.reset(tx, ty, tz, rx, ry, rz)
        else:
            # Rolling update without reallocation - see if we moved too much 
            txi, tyi, tzi, rxi, ryi, rzi = self.get_values(self.index-1)
            d_translation = np.sqrt((tx-txi)**2 + (ty-tyi)**2 + (tz-tzi)**2)
            d_rotation = np.sqrt((rx-rxi)**2 + (ry-ryi)**2 + (rz-rzi)**2)
            if d_translation > self.max_translation or d_rotation > self.max_rotation:
                # we're probably moving, so reset
                self.reset(tx, ty, tz, rx, ry, rz)
            else:
                for key in self.history:
                    self.history[key][self.index % self.max_averages] = new_values[["tx", "ty", "tz", "rx", "ry", "rz"].index(key)]
                self.index += 1
        # Compute averages ignoring NaN values
        averages = np.array([np.nanmean(self.history[key]) for key in self.history])

        # Compute standard deviation and check stability
        self.std_devs = np.array([np.nanstd(self.history[key]) for key in self.history])
        self.is_stable = np.all(self.std_devs < self.max_std)

        self.previous_time = self.current_time
        return tuple(map(float, averages))

class TagManager:
    def __init__(self, max_dt=1.0, max_averages=10, max_std=0.1):
        self.filters = {tag_id: TagFilter(max_dt, max_averages, max_std) for tag_id in range(1, 23)}
    
    def update(self, tag_id, tx, ty, tz, rx, ry, rz):
        if tag_id in self.filters:
            return self.filters[tag_id].update(tx, ty, tz, rx, ry, rz)
        else:
            raise ValueError(f"Invalid tag ID: {tag_id}")


In [8]:
# Example Usage

def n(sigma):
    return round(np.random.normal(0,sigma), 3)
    
tag_manager = TagManager(max_dt=0.1)

# Update tag 5 multiple times - averaging
print("\nTesting Tag 5: vanilla")
tag_id = 5
for i in range(1,5):
    print(tag_manager.filters[tag_id].index, tag_manager.update(tag_id, i * 0.1, i * 0.2, i * 0.3, i * 0.01, i * 0.02, i * 0.03))
    time.sleep(0.02)  # Simulate time delay

# Update tag 4 multiple times - no averaging
print("\nTesting Tag 4: growing dt")
tag_id = 4
for i in range(1,20):
    print(tag_manager.filters[tag_id].index, tag_manager.update(tag_id, i * 0.1, i * 0.2, i * 0.3, i * 0.01, i * 0.02, i * 0.03))
    time.sleep(0.008*(i%15))  # Simulate time delay

# Update tag 5 multiple times - averaging a random noise
print("\nTesting Tag 6:")
tag_id = 6
for i in range(1,15):
    std = 0.4
    data = 1+n(std), 2+n(std), 3+n(std), 4+n(std), 5+n(std),6+n(std)
    print(tag_manager.filters[tag_id].index, tag_manager.update(tag_id, *data) )
    time.sleep(0.02)  # Simulate time delay



Testing Tag 5: vanilla
0 (0.1, 0.2, 0.3, 0.01, 0.02, 0.03)
1 (0.15000000000000002, 0.30000000000000004, 0.44999999999999996, 0.015, 0.03, 0.045)
2 (0.20000000000000004, 0.4000000000000001, 0.6, 0.02, 0.04, 0.06)
3 (0.25, 0.5, 0.7499999999999999, 0.025, 0.05, 0.075)

Testing Tag 4: growing dt
0 (0.1, 0.2, 0.3, 0.01, 0.02, 0.03)
1 (0.15000000000000002, 0.30000000000000004, 0.44999999999999996, 0.015, 0.03, 0.045)
2 (0.20000000000000004, 0.4000000000000001, 0.6, 0.02, 0.04, 0.06)
3 (0.25, 0.5, 0.7499999999999999, 0.025, 0.05, 0.075)
4 (0.3, 0.6, 0.9, 0.030000000000000006, 0.06000000000000001, 0.09)
5 (0.35000000000000003, 0.7000000000000001, 1.0499999999999998, 0.035, 0.07, 0.10499999999999998)
6 (0.4, 0.8, 1.2, 0.04, 0.08, 0.11999999999999998)
7 (0.45, 0.9, 1.3499999999999999, 0.045, 0.09, 0.13499999999999998)
8 (0.5, 1.0, 1.4999999999999998, 0.049999999999999996, 0.09999999999999999, 0.15)
9 (0.55, 1.1, 1.65, 0.05499999999999999, 0.10999999999999999, 0.16499999999999998)
10 (0.65, 1.3,

In [13]:
tag_manager.filters[4].std_devs[0]

np.float64(0.14142135623730953)